# Data Visualization

## Libraries

In [103]:
from preprocessing import load_and_prepare_model, temperature_scaled_softmax
import random
import os
import torch
from ultralytics.utils import ops
import numpy as np
from ultralytics.utils.tal import dist2bbox

# Taxon Class

In [104]:
class Taxon:
    def __init__(self, name, rank):
        self.name = name
        self.rank = rank
        self.parent = None
        self.children = []

    def set_parent(self, parent):
        """Sets the parent of this taxon and adds this taxon as a child to the parent."""
        self.parent = parent
        parent.add_child(self)

    def add_child(self, child):
        """Adds a child taxon to this taxon."""
        self.children.append(child)

    def __str__(self):
        """String representation of the taxon."""
        return f"{self.rank}: {self.name}"

class TaxonomyManager:
    def __init__(self):
        self.taxa = {}  # Stores all taxa regardless of rank

    def add_taxon(self, name, rank, parent_name=None):
        """Adds a taxon to the taxonomy."""
        taxon = Taxon(name, rank)
        self.taxa[name] = taxon
        if parent_name:
            parent = self.taxa.get(parent_name)
            if parent:
                taxon.set_parent(parent)

    def get_taxon(self, name):
        """Retrieves a taxon by its name."""
        return self.taxa.get(name)

    def display_taxonomy(self):
        """Displays the entire taxonomy in a hierarchical format."""
        # Find roots (taxa without parents)
        roots = [taxon for taxon in self.taxa.values() if taxon.parent is None]
        for root in roots:
            self._display_taxon(root, 0)

    def _display_taxon(self, taxon, indent):
        """Recursive helper function to display a taxon and its children."""
        print(' ' * indent + str(taxon))
        for child in taxon.children:
            self._display_taxon(child, indent + 4)

# Populate Taxonomy

In [105]:
taxonomy = TaxonomyManager()

# Populate Objects
object_names = ["marine life", "inanimate"]
for name in object_names:
    taxonomy.add_taxon(name, "Binary")

# Populate Binaries
binary_mapping = {
    "asteroidea": "marine life",
    "phaeophyceae": "marine life",
    "bivalia": "marine life",
    "myxini": "marine life",
    "artificial": "inanimate",
    "natural": "inanimate",
    "chlorophyta": "marine life",
    "monocots": "marine life"
}
for class_name, binary_name in binary_mapping.items():
    taxonomy.add_taxon(class_name, "Class", binary_name)

# Populate Classes
class_mapping = {
    "asterias": "asteroidea",
    "fucus": "phaeophyceae",
    "henrica": "asteroidea",
    "mya": "bivalia",
    "myxine": "myxini",
    "cylindrical": "artificial",
    "solid": "natural",
    "arboral": "natural",
    "saccharina": "phaeophyceae",
    "ulva": "chlorophyta",
    "urospora": "chlorophyta",
    "zostera": "monocots"
}
for genus_name, class_name in class_mapping.items():
    taxonomy.add_taxon(genus_name, "Genus", class_name)

# Populate Genera
genus_mapping = {
    "asterias rubens": "asterias",
    "asteroidea": "asterias",
    "fucus vesiculosus": "fucus",
    "henrica": "henrica",
    "mytilus edulis": "mya",
    "myxine glurinosa": "myxine",
    "pipe": "cylindrical",
    "rock": "solid",
    "saccharina latissima": "saccharina",
    "tree": "arboral",
    "ulva intestinalis": "ulva",
    "urospora": "urospora",
    "zostera marina": "zostera"
}
for species_name, genus_name in genus_mapping.items():
    taxonomy.add_taxon(species_name, "Species", genus_name)

# Model Hooks

# Extract Prediction from Yolo v8

In [106]:
IMGDIR_PATH = "/mnt/RAID/datasets/label-studio/fjord/images/"
imgpath = IMGDIR_PATH + random.choice(os.listdir(IMGDIR_PATH))
model, hooks = load_and_prepare_model("runs/segment/Yolov8n-seg-train/weights/best.pt")

In [107]:
# Unpack hooks from load_and_prepare_model()
input_hook, detect, detect_hook, cv2_hooks, cv3_hooks = hooks

# Run inference
results = model(imgpath)

# Reverse engineer the outputs to find the logits
shape = detect_hook.input[0][0].shape  # BCHW
x = []
for i in range(detect.nl):
    x.append(torch.cat((cv2_hooks[i].output, cv3_hooks[i].output), 1))
x_cat = torch.cat([xi.view(shape[0], detect.no, -1) for xi in x], 2)
box, cls = x_cat.split((detect.reg_max * 4, detect.nc), 1)

# Batch size assumed to be 1
batch_idx = 0
dbox = dist2bbox(detect.dfl(box), detect.anchors.unsqueeze(0), xywh=True, dim=1) * detect.strides
cls = cls[batch_idx].sigmoid()

# Figure out the original img shape and model img shape to transform the boxes
img_shape = input_hook.input[0].shape[2:]
orig_img_shape = model.predictor.batch[1][batch_idx].shape[:2]

# Compute predictions
boxes = []
for i in range(dbox.shape[2]):  # Iterate over the third dimension (num_boxes)
    x0, y0, x1, y1 = dbox[0, :, i]  # Access all 4 coordinates for the i-th box
    x0, y0, x1, y1 = ops.scale_boxes(img_shape, np.array([x0.cpu(), y0.cpu(), x1.cpu(), y1.cpu()]), orig_img_shape)
    boxes.append({
        'image_id': imgpath,
        'bbox': [x0.item(), y0.item(), x1.item(), y1.item()],  # xyxy
        'logits': cls[:, i].tolist()
    })

# Prepare boxes for NMS
boxes_for_nms = torch.stack([
    torch.tensor([*b['bbox'], *b['logits']]) for b in boxes
], dim=1).unsqueeze(0)

nms_results = ops.non_max_suppression(boxes_for_nms, conf_thres=0.25, iou_thres=0.7)


image 1/1 /mnt/RAID/datasets/label-studio/fjord/images/frame005273.jpg: 384x640 1 urospora, 19.2ms
Speed: 3.4ms preprocess, 19.2ms inference, 20.7ms postprocess per image at shape (1, 3, 384, 640)


In [108]:
results[0].boxes[0]

ultralytics.engine.results.Boxes object with attributes:

cls: tensor([11.], device='cuda:0')
conf: tensor([0.8621], device='cuda:0')
data: tensor([[9.6964e+02, 3.5041e+01, 1.5321e+03, 1.6818e+02, 8.6212e-01, 1.1000e+01]], device='cuda:0')
id: None
is_track: False
orig_shape: (1080, 1920)
shape: torch.Size([1, 6])
xywh: tensor([[1250.8929,  101.6094,  562.5090,  133.1366]], device='cuda:0')
xywhn: tensor([[0.6515, 0.0941, 0.2930, 0.1233]], device='cuda:0')
xyxy: tensor([[ 969.6385,   35.0411, 1532.1475,  168.1776]], device='cuda:0')
xyxyn: tensor([[0.5050, 0.0324, 0.7980, 0.1557]], device='cuda:0')

In [109]:
nms_results[0]

tensor([[9.6964e+02, 5.3041e+01, 1.5321e+03, 1.5018e+02, 8.6212e-01, 1.1000e+01]])